# Atividade 1 - Ciência dos Dados
1 - Efetuar a extração de 500 cursos de pós-graduação do link https://sucupira.capes.gov.br/programas/detalhamento/2099, dica, analise o site para se obter todo o conteúdo. De cada curso você deve extrair: código, areaBasica, areaAvaliacao, situação, cidade, mestrado, nota, situacaoMestrado, doutorado, codigoMestrado, situacaoDoutorado, notaMestrado, codigoDoutorado, cep, início e universidade.

In [ ]:
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
import time
import re
import csv

curso = []
n = 1
while len(curso) < 50: # mudar para extrair mais informações
    url = "https://sucupira.capes.gov.br/programas/detalhamento/{}".format(1500 + n)
    print(url)
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Edge(options=options)
    driver.get(url)
    time.sleep(3)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    if re.search(r'(?=.*Mestrado)(?=.*Doutorado)', soup.find('div', class_='row bg-cinza0 my-2').text):
        div_q_item_label = soup.find_all('div', class_='q-item__label')

        for i, text in enumerate(div_q_item_label[:-8]):
            if i == 1:
                resultado = re.search(r'\((.*?)\).*Nota\s+(\d+)', text.text)
                if resultado:
                    codigo = resultado.group(1)  
                    nota = resultado.group(2)    
            if i == 9:
                area_basica = text.text
            if i == 7:
                area_avaliacao = text.text
            if i == 15:
                situacao = text.text

        div_lista = soup.find_all('div', class_='item-lista')
        for i, item in enumerate(div_lista):
            span_tag = item.find('span')
            if span_tag:
                if i == 0:
                    universidade = span_tag.get_text()

        h6 = soup.find_all('h6')
        for item in h6:
            texto = item.get_text(strip=True)
            if re.search(r'.*\sMestrado', texto):
                mestrado = texto
            if re.search(r'.*\sDoutorado', texto):
                doutorado = texto  

        info_mestr = soup.find(string=mestrado).find_next()
        for item in info_mestr:
            nota_mestrado = item.find('div', class_='q-badge').text
            cod_mestrado = item.find_all('span')[0].strong.text
            inicio_mestrado = item.find_all('span')[2].strong.text
            sit_mestrado = item.find_all('span')[3].text

        info_dout = soup.find(string=doutorado).find_next()
        for item in info_dout:
            nota_doutorado = item.find('div', class_='q-badge').text
            cod_doutorado = item.find_all('span')[0].strong.text
            inicio_doutorado = item.find_all('span')[2].strong.text
            sit_doutorado = item.find_all('span')[3].text

        curso.append([codigo, nota, area_basica, area_avaliacao, situacao, universidade, mestrado, nota_mestrado, cod_mestrado, inicio_mestrado, sit_mestrado, doutorado, nota_doutorado, cod_doutorado, inicio_doutorado, sit_doutorado])
    
    n+=1
    driver.quit()

with open('pos_graduacao.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['codigo', 'nota', 'area_basica', 'area_avaliacao', 'situacao', 'universidade', 'mestrado', 'nota_mestrado', 'cod_mestrado', 'inicio_mestrado', 'sit_mestrado', 'doutorado', 'nota_doutorado', 'cod_doutorado', 'inicio_doutorado', 'sit_doutorado'])
    writer.writerows(curso)

2- Efetuar a extração de duas urls diferentes através da linguagem Python, e ter como saída um documento dois csv contendo todos os dados extraídos. Submeter o código em Python

**Url 1**

https://books.toscrape.com/

Campos extraídos:
   - nome
   - preço
   - rating
   - disponibilidade

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
import csv

produtos = []
n = 1
while n <= 5:
    url = 'https://books.toscrape.com/catalogue/page-{}.html'.format(n) 
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Edge(options = options)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html)
    product_pod = soup.find_all('article', class_='product_pod')
    for item in product_pod:
        nome = item.img['alt']
        preco = item.find('p', class_='price_color').text
        disp = item.find('p', class_='instock availability').text.strip()
        rating = item.find('p', class_='star-rating')['class'][1]
        livro = [nome, preco, rating, disp]
        produtos.append(livro)
    n+=1
    driver.quit()

with open('produtos.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['nome', 'preço', 'rating', 'disponibilidade'])
    writer.writerows(produtos)

**Url 2**

https://quotes.toscrape.com/

Campos extraídos:

  - quote
  - autor
  - tags

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options
import csv

conteudos = []
n = 1
while n <= 5:
    url = 'https://quotes.toscrape.com/page/{}'.format(n)
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Edge(options = options)
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html)
    quotes = soup.find_all('div', class_='quote')
    for item in quotes:
        quote = item.find('span', class_='text').text
        author = item.find('small', class_='author').text
        tags = item.find('meta', class_='keywords')['content']
        conteudo = [quote, author, tags]
        conteudos.append(conteudo)
    n+=1
    driver.quit()

with open('quotes.csv', mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['quote', 'author', 'tags'])
    writer.writerows(conteudos)

3 - Os blogs e outros sites atualizados regularmente em geral têm uma página inicial com a postagem mais recente, além de um botão Previous (Anterior) na página que conduzirá você à postagem anterior. Então essa postagem também terá um botão Previous e assim sucessivamente, criando um percurso que conduz da página mais recente até a primeira postagem do site. Se quiser uma cópia do conteúdo do site para ler quando não estiver online, você poderá navegar manualmente por todas as páginas e salvar cada uma delas. Contudo esse é um trabalho bem maçante, portanto vamos criar um programa que faça isso. O XKCD é um webcomic popular para geeks com um site que se enquadra nessa estrutura. A página inicial em http://xkcd.com/ contém um botão Prev (Anterior) que conduz o usuário às tirinhas anteriores. Fazer download de cada tirinha manualmente demoraria muito tempo, porque não utilizarmos o conteúdo aprendido nesta aula? Seu programa deve fazer:

- Carregar a página inicial de XKCD.
- Salvar a imagem da tirinha que está nessa página.
- Seguir o link para Previous Comic (Tirinha anterior).
- Repetir até alcançar a primeira tirinha.

Isso significa que seu código deverá fazer o seguinte:

- Fazer download de páginas com o módulo requests.
- Encontrar o URL da imagem da tirinha em uma página usando o BeautifulSoup.
- Fazer download e salvar a imagem da tirinha no disco rígido. Dica de função a ser utilizada para salvar as tirinhas: iter_content().
- Encontrar o URL do link Previous Comic (Tirinha anterior) e repetir

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

url = 'http://xkcd.com/'
os.makedirs('images', exist_ok=True)

# para fazer o download de todas as tirinhas já publicadas basta substitir o loop por:
# while not url.endswith('#')
n = 0
while n <= 49: 
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    comic_src = soup.find('div', id='comic').img['src']
    comic_url = f"http:{comic_src}"
    image_response = requests.get(comic_url)
    image_response.raise_for_status()
    filename = comic_url.split('/')[-1]
    with open('images/' + filename, 'wb') as file:
        for content in image_response.iter_content(100000):
            file.write(content)

    previous = soup.find('a', rel='prev')
    url = f"http://xkcd.com{previous['href']}"
    n+=1